In [1]:
import pandas as pd
import geopandas as gpd
from shapely import Point
from math import radians, cos, sin, asin, sqrt
import dask.dataframe as dd
from pathlib import Path
import sys, os
sys.path.append(os.path.join(os.path.abspath(""), '..', 'pipeline'))

In [2]:
import distances
import counterglow_matches as cg

In [3]:
# Enter the SIC Code of interest here
SIC_CODE = '0291'

# radius that is used as a limit of distance between two sets of coordinates
RADIUS_KM = 0.3048

## Functions & File Load In

In [60]:
# function to calculuate distance between 2 sets of coordinates

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    # print("Distance:", c * r)
    return c * r

In [61]:
"""
funtion will take 4 inputs. infogroup_df & counterglow_df2 are the dataframes being analyzed.
the third input is the specific state that we are choosing to analyze
the fourth input is the threshold distance. this distance is the max allowable distance between 2
datapoints. the output is a dataframe that in each row, we have a datapoint that represents a
datapoint from infogroup and (x distance away) a farm spotted in the counterglow data. we are
speculating that it is possible for a spotted farm in counterglow to not have the most precise address
and actually exist in the Infogroup data, but with slightly different longitude and latitude
"""

def potential_farms(infogroup_df, counterglow_df, two_letter_state, max_dist_km ):
    
    # ruduce dataframe to only consist rows with specified two_letter_state
    infogroup_state_df      = infogroup_df[infogroup_df["STATE"] == two_letter_state]
    counterglow_state_df    = counterglow_df[counterglow_df["State"] == two_letter_state]
    
    # create two new empty dataframe that we will be adding to when match is found
    infogroup_trim     = pd.DataFrame(columns=infogroup_state_df.columns)
    counterglow_trim   = pd.DataFrame(columns=counterglow_state_df.columns)
    
    counter = 0 # to counter matches, can print to terminal if desired
    for i in range(len(infogroup_state_df)):
        
        infogroup_longitude  = infogroup_state_df.iloc[i]["LONGITUDE"]  # get longitude
        infogroup_latitude   = infogroup_state_df.iloc[i]["LATITUDE"]   # get latitude
        
        for j in range(len(counterglow_state_df)):
            
            counterglow_latitude    = counterglow_state_df.iloc[j]["Lat"]    # get latitude
            counterglow_longitude   = counterglow_state_df.iloc[j]["Lat.1"]  # get longitude
            
            dist_km                 = haversine(infogroup_longitude, infogroup_latitude, 
                                                counterglow_longitude, counterglow_latitude)
            
            if(dist_km <= max_dist_km): # if the distance is <= the max distance set in function
                counter += 1
            
                infogroup_trim.loc[len(infogroup_trim.index)]      = infogroup_state_df.iloc[i]
                counterglow_trim.loc[len(counterglow_trim.index)]  = counterglow_state_df.iloc[j]
        
    # change the name of the columns for Latitude & Longitude in each dataframe
    # for Infogroup Dataframe
    infogroup_trim      = infogroup_trim.rename(columns={'LATITUDE': 'LATITUDE 1', 
                                                        'LONGITUDE': 'LONGITUDE 1'})
    counterglow_trim    = counterglow_trim.rename(columns={'Lat' : 'LATITUDE 2',
                                                           'Lat.1': 'LONGITUDE 2'})
    
    # print to terminal to let user know there were matches found
    if(counter !=0):
        print(two_letter_state + ":", "Total number of matches:", counter)
                
    return infogroup_trim, counterglow_trim   
    

In [62]:
"""
function take a dataframe and a list of states. This function calls the function 'potential_farms'
if the two dataframes returned from 'potential_farms' are NOT empty, they are added to the dictionaries
the keys for the dictionaries are the STATE in the current iteration (in the for loop) and the values
are the dataframes associated with the state. This function will return two dictionaries

This function POTENTIAL_FARMS function
"""

def infogroup_counterglow_dict(sic_df, counterglow_df, list_of_states, radium_km):
    # empty dictionaries
    igroup_sic_dict = {}
    ctrglow_dict = {}

    # loop through states list and use those as keys to match with the dataframe that was output 
    for state in range(len(list_of_states)):
        infogroup_matches, counterglow_matches = potential_farms(sic_df, counterglow_df, list_of_states[state], radium_km)
        
        # set up key value pairing for both dictionaries
        # only adding to dictionaries if there are matches
        if( (len(infogroup_matches)!=0) & (len(counterglow_matches)!=0) ):
            igroup_sic_dict[list_of_states[state]]    = infogroup_matches
            ctrglow_dict[list_of_states[state]]       = counterglow_matches
            
    return igroup_sic_dict, ctrglow_dict


In [63]:
"""
fucntion will take a dataframe and trim off all columns, only leaving:
Company, address line 1, city, state, zipcode, primary sic code, sic code 1, sic code 2, sic code 3, sic code 4,
"""

def keep_certain_columns(dict):
    for key in dict:
        dict[key] = dict[key][['COMPANY', 'ADDRESS LINE 1', 'CITY', 'STATE', 'ZIPCODE', 'PRIMARY SIC CODE', 'SIC6_DESCRIPTIONS',
                'PRIMARY NAICS CODE', 'NAICS8 DESCRIPTIONS', 'SIC CODE', 'SIC6_DESCRIPTIONS (SIC)',
                'SIC CODE 1', 'SIC6_DESCRIPTIONS (SIC1)', 'SIC CODE 2', 'SIC6_DESCRIPTIONS(SIC2)',
                'SIC CODE 3', 'SIC6_DESCRIPTIONS(SIC3)', 'SIC CODE 4', 'SIC6_DESCRIPTIONS(SIC4)', 'PARENT NUMBER']]
    
    return dict


In [64]:
# function that take a dictionary, the keys are STATES (2 letter) and the VALUES are a dataframe
# containing the locations that are close to a counterglow location

def list_of_dataframes(infogroup_dict, master_list):
    
    # loop through each key in the dictinary (each state in this case)
    for key in infogroup_dict.keys():
        # drop the duplicate rows and then add the trimmed dataframe to master_list
        master_list.append(infogroup_dict[key].drop_duplicates())
        
    return master_list

## close

In [4]:
# establishing the file path in such a way to work universally
here = Path().resolve()

In [5]:
# load counterglow dataset
# df_counterglow = pd.read_csv("../data/Counterglow+Facility+List+Complete.csv")

df_counterglow = pd.read_csv(here.parent / "data/Counterglow+Facility+List+Complete.csv")

In [6]:
# filter counterglow dataset to have poultry/chicken only

# Chickens & Other Birds (Meat)
# Chickens (Meat)

df_counterglow_poultry = df_counterglow[(df_counterglow['Farm Type'] == 'Chickens & Other Birds (Meat)') |
                                        (df_counterglow['Farm Type'] == 'Chickens (Meat)')]

In [7]:
# list of states we want to observe
states = ['IA', 'OK', 'MO', 'OK', 'AL', 'LA', 'MS', 'IL', 'IN', 'OH', 'KY', 'TN', 'AR', 'NC', 'SC', 'GA']

# all the dataframes 
all_df_list = []

# subset of dataframe, SIC that we felt were most relevant
select_df_list = []

## SIC Code 0291 Exploration: General Farms, Primarily Livestock and Animal Specialties.

In [8]:
print(here)

C:\Users\chacon1\OneDrive - University of Illinois at Chicago\Desktop\rafi-poultry\notebooks


In [9]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing SIC_CODE of interest

path = "data/code_" + SIC_CODE + ".csv"
df_sic = pd.read_csv(here.parent / path)
  
# igroup_sic_0291_dict, ctrglow_0291_dict = infogroup_counterglow_dict(df_sic, df_counterglow_poultry, states, RADIUS_KM)
igroup_sic_0291_dict, ctrglow_0291_dict = cg.infogroup_counterglow_dict(df_sic, df_counterglow_poultry, states, RADIUS_KM)


NC: Total number of matches: 1


In [71]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_0291_dict, all_df_list)

In [102]:
select_df_list = list_of_dataframes(igroup_sic_0291_dict, select_df_list)

In [72]:
len(all_df_list)

1

#### Explore

In [ ]:
igroup_sic_0291_dict['AL']

In [17]:
igroup_sic_0291_dict['NC']['PRIMARY SIC CODE']

0    29101
Name: PRIMARY SIC CODE, dtype: object

## SIC Code 0241 Exploration: Dairy Farms

In [73]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0241
df_sic_0241 = pd.read_csv("../data/code_0241.csv")
  
igroup_sic_0241_dict, ctrglow_0241_dict = infogroup_counterglow_dict(df_sic_0241, df_counterglow_poultry, states, RADIUS_KM)


OH: Total number of matches: 1
NC: Total number of matches: 1


In [74]:
# igroup_sic_0241_dict = keep_certain_columns(igroup_sic_0241_dict)

In [75]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_0241_dict, all_df_list)

#### Explore

In [21]:
igroup_sic_0241_dict['IN']

type: 'IN'

In [23]:
igroup_sic_0241_dict['OH']['PRIMARY SIC CODE']

0    24103
Name: PRIMARY SIC CODE, dtype: object

In [25]:
igroup_sic_0241_dict['NC']['PRIMARY SIC CODE']

0    24103
Name: PRIMARY SIC CODE, dtype: object

## SIC Code 0761 Exploration: Farm Labor

In [76]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0761
df_sic_0761 = pd.read_csv("../data/code_0761.csv")
  
igroup_sic_0761_dict, ctrglow_0761_dict = infogroup_counterglow_dict(df_sic_0761, df_counterglow_poultry, states, RADIUS_KM)


In [77]:
# igroup_sic_0761_dict = keep_certain_columns(igroup_sic_0761_dict)

In [77]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_0761_dict, all_df_list)

#### Explore

## SIC Code 0762 Exploration: Farm Mgmt Services

In [78]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_0762 = pd.read_csv("../data/code_0762.csv")
  
igroup_sic_0762_dict, ctrglow_0762_dict = infogroup_counterglow_dict(df_sic_0762, df_counterglow_poultry, states, RADIUS_KM)


MS: Total number of matches: 1


In [29]:
# igroup_sic_0762_dict = keep_certain_columns(igroup_sic_0762_dict)

In [79]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_0762_dict, all_df_list)

#### Explore

In [32]:
igroup_sic_0762_dict['MS']['PRIMARY SIC CODE']

0    19101
Name: PRIMARY SIC CODE, dtype: object

In [ ]:
igroup_sic_0762_dict['OH']

## SIC Code 8748 Exploration: Farm Mgmt Systems

In [80]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 8748
df_sic_8748 = pd.read_csv("../data/code_8748.csv")
  
igroup_sic_8748_dict, ctrglow_8748_dict = infogroup_counterglow_dict(df_sic_8748, df_counterglow_poultry, states, RADIUS_KM)


In [25]:
# igroup_sic_8748_dict = keep_certain_columns(igroup_sic_8748_dict)

In [81]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_8748_dict, all_df_list)

#### Explore

In [ ]:
igroup_sic_8748_dict['IA']

In [ ]:
igroup_sic_8748_dict['MO']

In [ ]:
igroup_sic_8748_dict['AL']

In [ ]:
igroup_sic_8748_dict['MS']

In [ ]:
igroup_sic_8748_dict['IL']

In [ ]:
igroup_sic_8748_dict['IN']

In [ ]:
igroup_sic_8748_dict['AR']

In [ ]:
igroup_sic_8748_dict['NC']

## SIC Code 5431 Exploration: Farm Markets

In [82]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 5431
df_sic_5431 = pd.read_csv("../data/code_5431.csv")
  
igroup_sic_5431_dict, ctrglow_5431_dict = infogroup_counterglow_dict(df_sic_5431, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 1


In [35]:
# igroup_sic_5431_dict = keep_certain_columns(igroup_sic_5431_dict)

In [83]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_5431_dict, all_df_list)

#### Exploration

In [47]:
igroup_sic_5431_dict['IA']['PRIMARY SIC CODE']

0    25401
Name: PRIMARY SIC CODE, dtype: object

In [ ]:
igroup_sic_5431_dict['AR']

In [ ]:
igroup_sic_5431_dict['NC']

## SIC Code 8611 Exploration: Farm Organizations

In [84]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 8611
df_sic_8611 = pd.read_csv("../data/code_8611.csv")
  
igroup_sic_8611_dict, ctrglow_8611_dict = infogroup_counterglow_dict(df_sic_8611, df_counterglow_poultry, states, RADIUS_KM)


IL: Total number of matches: 1


In [40]:
# igroup_sic_8611_dict = keep_certain_columns(igroup_sic_8611_dict)

In [85]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_8611_dict, all_df_list)

In [103]:
select_df_list = list_of_dataframes(igroup_sic_8611_dict, select_df_list)

#### Exploration

In [ ]:
igroup_sic_8611_dict['MS']

In [55]:
igroup_sic_8611_dict['IL']['SIC6_DESCRIPTIONS (SIC)']

0    ASSOCIATIONS
Name: SIC6_DESCRIPTIONS (SIC), dtype: object

In [43]:
igroup_sic_8611_dict['AR']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,NORTHWEST ARK HM BLDRS ASSN,2022 LONG ST,SPRINGDALE,AR,72764,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SPRINGDALE CHAMBER OF COMMERCE,1200 S THOMPSON ST,SPRINGDALE,AR,72764,861104,CHAMBERS OF COMMERCE,81391005.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
igroup_sic_8611_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,QUEEN'S CUP STEEPLECHASE,6103 WAXHAW HWY,MINERAL SPRINGS,NC,28108,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIC Code 0723 Exploration: Farm Produce

In [86]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0723
df_sic_0723 = pd.read_csv("../data/code_0723.csv")
  
igroup_sic_0723_dict, ctrglow_0723_dict = infogroup_counterglow_dict(df_sic_0723, df_counterglow_poultry, states, RADIUS_KM)


In [47]:
# igroup_sic_0723_dict = keep_certain_columns(igroup_sic_0723_dict)

In [87]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_0723_dict, all_df_list)

#### Explore

## SIC Code 0191 Exploration: Farm Products

In [88]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0191
df_sic_0191 = pd.read_csv("../data/code_0191.csv")
  
igroup_sic_0191_dict, ctrglow_0191_dict = infogroup_counterglow_dict(df_sic_0191, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 13
MO: Total number of matches: 4
AL: Total number of matches: 9
MS: Total number of matches: 5
IL: Total number of matches: 1
IN: Total number of matches: 4
NC: Total number of matches: 5
SC: Total number of matches: 1


In [50]:
# igroup_sic_0191_dict = keep_certain_columns(igroup_sic_0191_dict)

In [89]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_0191_dict, all_df_list)

In [104]:
select_df_list = list_of_dataframes(igroup_sic_0191_dict, select_df_list)

#### Explore

In [51]:
igroup_sic_0191_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,KDD INC,4755 20TH ST,GRINNELL,IA,50112,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CENTRUM VALLEY FARMS,2975 WASHINGTON AVE,DOWS,IA,50071,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CENTURN VALLEY FARMS,2688 270TH ST,GALT,IA,50101,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CENTRUM VALLEY FARMS,2674 HIGHWAY 69,GALT,IA,50101,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),541105.0,GROCERS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DAVID HOAKISON FARMS,1089 JOSHUA AVE,CRESTON,IA,50801,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,FORSMAN FARMS,2975 265TH ST,VILLISCA,IA,50864,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DE COSTER FARMS OF IOWA,1795 DOGWOOD AVE,ALEXANDER,IA,50420,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SIOUX COUNTY EGG FARM LLP,2716 380TH ST,SIOUX CENTER,IA,51250,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,514404.0,POULTRY-WHOLESALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MERLIN DE GROOT FARM,4702 360TH ST,SHELDON,IA,51201,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BROWN FARMS,2810 480TH AVE,GRAETTINGER,IA,51342,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
igroup_sic_0191_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,JOHNSON COUNTY EGG FARM,1275 SE Y HWY,KNOB NOSTER,MO,65336,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JOHNSON COUNTY EGG FARM,1285 SE Y HWY,KNOB NOSTER,MO,65336,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DANNY MAZELIN FARM,33403 ALMOND RD,COLE CAMP,MO,65325,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RIVERAS FARMS LLC,1316 FARM ROAD 2155,EXETER,MO,65647,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MOARK PRODUCTION INC,19242 MALLARD DR,NEOSHO,MO,64850,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,GM FARMS,17684 WALLEYE RD,FAIRVIEW,MO,64842,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ARNAUD VICTOR FARMS,7457 FARM ROAD 2040,MONETT,MO,65708,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DOCMO FARM,9988 COUNTY ROAD 488,DUDLEY,MO,63936,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DICK MCKUIN FARMS,2320 STATE HIGHWAY 25,BERNIE,MO,63822,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
igroup_sic_0191_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,NUNLEY FARMS LLC,12527 HIGHWAY 23,VINA,AL,35593,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WENDELL GIBBS FARMS,2118 COUNTY ROAD 23,RANBURNE,AL,36273,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C J FARMS,486 COUNTY ROAD 95,RANBURNE,AL,36273,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C J FARMS,486 COUNTY ROAD 95,RANBURNE,AL,36273,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GRANBERRY FARM,5955 COUNTY ROAD 55,COLUMBIA,AL,36319,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BAILEY FARMS,3536 SARDIS RD,ASHLAND,AL,36251,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GIBBS FARMS,1358 COUNTY ROAD 68,RANBURNE,AL,36273,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SOUTH HENRY PEANUTS INC,NaN,HEADLAND,AL,36345,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,HUGULEY FARMS INC,5405 N COUNTY ROAD 65,COFFEE SPRINGS,AL,36318,78203,SOD & SODDING SERVICE,56173019.0,LANDSCAPING SERVICES,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TRIPLE O FARMS LLC,643 GILBERT RD,LOWNDESBORO,AL,36752,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
igroup_sic_0191_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TCL POULTRY FARM,83 COURTNEY LN,BASSFIELD,MS,39421,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ROWLEY FARMS INC,313 ROWLEY RD,FOXWORTH,MS,39483,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B B FARMS,667 LOWER BROOKHAVEN RD,MONTICELLO,MS,39654,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MARTIN FARMS LLC,12302 HIGHWAY 98,SMITHDALE,MS,39664,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOMOCHITTO POULTRY FARM,3118 CASEYVILLE RD NW,WESSON,MS,39191,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M & N FARMS,2868 SHILOH RD # A,PELAHATCHIE,MS,39145,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ALYSSA FARM LLC,1140 LAWRENCE CONEHATTA RD,LAWRENCE,MS,39336,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,GREAT SOUTHERN FARMS LLC,1729 HIGHWAY 15,RICHTON,MS,39476,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,STUBBS FARMS,129 MILITARY RD,SUMRALL,MS,39482,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,J & D'S FARMS,964 TOWNSEND DR SE,BROOKHAVEN,MS,39601,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
igroup_sic_0191_dict['IL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HI-GRADE EGG,317 E 360 NORTH RD # 2,LODA,IL,60948,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
igroup_sic_0191_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,ISCH FARMS INC,1977 S 400 W-1,BLUFFTON,IN,46714,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S & G POULTRY FARM,2496 S 400 W,MONROE,IN,46772,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,J & K TURKEY FARM LLC,4329 E 225 S,WASHINGTON,IN,47501,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PERDUE FARMS INC,500 PERDUE RD,VINCENNES,IN,47591,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,201501.0,POULTRY PROCESSING PLANTS (MFRS),25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,7529217.0
4,ROSE ACRE FARMS INC,5438 W STATE ROAD 16,RENSSELAER,IN,47978,25298,CHICKEN EGGS,11231001.0,CHICKEN EGG PRODUCTION,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1081025.0
5,THOMAS BONTRAGER FARM,63666 COUNTY ROAD 37,GOSHEN,IN,46528,19198,GENERAL FARMS-PRIMARILY CROP,11199807.0,ALL OTHER MISCELLANEOUS CROP FARMING,24103.0,DAIRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,JONES FARMS,12210 N BRADFORD RD NE,PALMYRA,IN,47164,19198,GENERAL FARMS-PRIMARILY CROP,11199807.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MIDWEST PULLET FARMS,4854 S 1000 W,MENTONE,IN,46539,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MID RIVER FARMS,10335 COUNTY ROAD 18,MIDDLEBURY,IN,46540,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BROVONT SOW FARM INC,2322 W STATE ROAD 14,SILVER LAKE,IN,46982,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
igroup_sic_0191_dict['OH']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HERTZFELD POULTRY FARMS,15799 MILTON RD,GRAND RAPIDS,OH,43522,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOMAN FARMS,6671 ASHMAN RD,MARIA STEIN,OH,45860,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,13901.0,AGRICULTURAL PRODUCTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
igroup_sic_0191_dict['TN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SUNSET RIDGE CATTLE CO,400 BRODIO RD,DEER LODGE,TN,37726,75102,LIVESTOCK BREEDERS,11521010.0,SUPPORT ACTIVITIES FOR ANIMAL PRODUCTION,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
igroup_sic_0191_dict['AR']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,U-PICK IT FARM & GARDEN CORP,17369 WYOLA RD,WINSLOW,AR,72959,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
igroup_sic_0191_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,LEONARD LAKES FARMS,329 LEONARD RD,LINCOLNTON,NC,28092,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CONYERS' FARMS,8520 COLLIERS CHAPEL CHURCH RD,LINDEN,NC,28356,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WHITESIDES FARM & RENTAL LLC,3287 JUNE BUG RD,VALE,NC,28168,19198,GENERAL FARMS-PRIMARILY CROP,11199807.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ROCK CREEK FARM,7014 MOUNT HERMON ROCK CRK RD,SNOW CAMP,NC,27349,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ROCK CREEK FARM,7014 MOUNT HERMON ROCK CRK RD,SNOW CAMP,NC,27349,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MITCHELL FARMS,4167 CHURCH RD,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MITCHELL FARMS,4167 CHURCH RD,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MITCHELL FARMS,4167 CHURCH RD,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MITCHELL FARMS,4167 CHURCH RD,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,FARM GRAND C,298 R AND R LN,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
igroup_sic_0191_dict['SC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,WILSON RIDGE FARM LLC,5116 WYLIES MILL RD,EDGEMOOR,SC,29712,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BELGER FARMS & CONSTR INC,2044 BOYKIN RD,REMBERT,SC,29128,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
igroup_sic_0191_dict['GA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HOUSE OF RAEFORD FARMS INC,55 COOPER RD,COMER,GA,30629,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513523.0


## SIC Code 254 Exploration: Poultry Farms

In [90]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 254
df_sic_254 = pd.read_csv("../data/code_254.csv")
  
igroup_sic_254_dict, ctrglow_254_dict = infogroup_counterglow_dict(df_sic_254, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 5
MO: Total number of matches: 2
AL: Total number of matches: 3
MS: Total number of matches: 1
IN: Total number of matches: 2
AR: Total number of matches: 1
GA: Total number of matches: 3


In [68]:
# igroup_sic_254_dict = keep_certain_columns(igroup_sic_254_dict)

In [91]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_254_dict, all_df_list)

In [105]:
select_df_list = list_of_dataframes(igroup_sic_254_dict, select_df_list)

#### Explore

In [69]:
igroup_sic_254_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,VALO BIOMEDIA NORTH AMERICA,21459 OLD HIGHWAY 6,ADEL,IA,50003,283402,POULTRY MEDICINES & VACCINES (MFRS),32541208.0,PHARMACEUTICAL PREPARATION MANUFACTURING,25401.0,POULTRY FARMS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,545101.0,DAIRY PRODUCTS-RETAIL,514404.0,POULTRY-WHOLESALE,NaN,NaN,843668294.0
1,VALO BIOMEDIA NORTH AMERICA,16332 OLD HIGHWAY 6,REDFIELD,IA,50233,283402,POULTRY MEDICINES & VACCINES (MFRS),32541208.0,PHARMACEUTICAL PREPARATION MANUFACTURING,873104.0,LABORATORIES,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,843668294.0
2,CALEKO FARM INC,3819 330TH ST,BOYDEN,IA,51234,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,REMBRANDT ENTERPRISES,4252 80 AVE,LINN GROVE,IA,51033,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,REMBRANDT ENTERPRISES,4252 80 AVE,LINN GROVE,IA,51033,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,IOWA CAGE FREE LLP,1035 200TH ST,GOLDFIELD,IA,50542,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,FARMERS FRESH POULTRY,5195 FARMERS AVE SW,KALONA,IA,52247,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,543102.0,FARM MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,RIVERLAND POULTRY,2015 HIGHWAY 22,KALONA,IA,52247,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
igroup_sic_254_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,LEUSCHEN POULTRY OPERATIONS,1175 DAIRY CIR,CRANE,MO,65633,514402,POULTRY SERVICES (WHLS),42444002.0,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RIVERAS FARMS LLC,1316 FARM ROAD 2155,EXETER,MO,65647,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DICK MCKUIN FARMS,2320 STATE HIGHWAY 25,BERNIE,MO,63822,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
igroup_sic_254_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,ROCK CREEK FARMS,400 COUNTY ROAD 134,TOWN CREEK,AL,35672,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,19101.0,FARMS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FOUR H FARMS,2440 COUNTY ROAD 113,CENTRE,AL,35960,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BREEZY BOTTOM FARM,158 NICHOLS RD,ALTOONA,AL,35952,421304,TRUCKING,48423013.0,SPECIALIZED FREIGHT (EXC USED GDS) TRCKNG LNG-...,19101.0,FARMS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DALE MURPHY POULTRY,1355 WARRIOR VALLEY RD,ALTOONA,AL,35952,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
igroup_sic_254_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,EVANS POULTRY FARMS,380 SCR 79,MIZE,MS,39116,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
igroup_sic_254_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,EGG INNOVATIONS,4811 W 100 N,WARSAW,IN,46580,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RIVER VIEW FARMS INC,2262 E COUNTY ROAD 500 N,ORLEANS,IN,47452,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,S & G POULTRY FARM,2496 S 400 W,MONROE,IN,46772,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PERDUE FARMS INC,500 PERDUE RD,VINCENNES,IN,47591,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,201501.0,POULTRY PROCESSING PLANTS (MFRS),25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,7529217.0
4,MINNICH POULTRY,8563 E 300 N,PORTLAND,IN,47371,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
igroup_sic_254_dict['OH']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HERTZFELD POULTRY FARMS,15799 MILTON RD,GRAND RAPIDS,OH,43522,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MEYER HATCHERY,626 STATE ROUTE 89,POLK,OH,44866,92101,FISHERY CONSULTANTS,11251102.0,FINFISH FARMING & FISH HATCHERIES,25401.0,POULTRY FARMS,549907.0,POULTRY-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
igroup_sic_254_dict['AR']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,GEORGE'S INC,402 W ROBINSON AVE,SPRINGDALE,AR,72764,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,512392.0


In [78]:
igroup_sic_254_dict['GA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TYSON FOODS INC,1151 E PINE ST,VIENNA,GA,31092,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,514101.0,FOOD PRODUCTS (WHLS),209903.0,FOOD PRODUCTS & MANUFACTURERS,549907.0,POULTRY-RETAIL,549911.0,GAME BIRDS VENISON ETC,25401.0,POULTRY FARMS,7537913.0
1,CLAXTON POULTRY FARMS,8816 US HIGHWAY 301,CLAXTON,GA,30417,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,25401.0,POULTRY FARMS,509901.0,EXPORTERS (WHLS),514404.0,POULTRY-WHOLESALE,514403.0,EGGS (WHLS),NaN,NaN,NaN
2,PILGRIM'S PRIDE CORP,136 SWAN BRIDGE RD,TALKING ROCK,GA,30175,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,201501.0,POULTRY PROCESSING PLANTS (MFRS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,987289857.0


## SIC Code 5144 Exploration: Poultry & Poultry Products

In [92]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 5144
df_sic_5144 = pd.read_csv("../data/code_5144.csv")
  
igroup_sic_5144_dict, ctrglow_5144_dict = infogroup_counterglow_dict(df_sic_5144, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 9
MO: Total number of matches: 1
KY: Total number of matches: 1
NC: Total number of matches: 1
GA: Total number of matches: 1


In [80]:
# igroup_sic_5144_dict = keep_certain_columns(igroup_sic_5144_dict)

In [93]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_5144_dict, all_df_list)

In [106]:
select_df_list = list_of_dataframes(igroup_sic_5144_dict, select_df_list)

#### Explore

In [81]:
igroup_sic_5144_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,VALO BIOMEDIA NORTH AMERICA,21459 OLD HIGHWAY 6,ADEL,IA,50003,283402,POULTRY MEDICINES & VACCINES (MFRS),32541208,PHARMACEUTICAL PREPARATION MANUFACTURING,25401.0,POULTRY FARMS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,545101.0,DAIRY PRODUCTS-RETAIL,514404.0,POULTRY-WHOLESALE,NaN,NaN,843668294.0
1,FREMONT FARMS OF IOWA,3868 110TH ST,MALCOM,IA,50157,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CENTRUM VALLEY FARMS,2674 HIGHWAY 69,GALT,IA,50101,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),541105.0,GROCERS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IOWA CAGE FREE,1641 YELLOWSTONE AVE,CLEARFIELD,IA,50840,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,541105.0,GROCERS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OSAGE EGG CO,3405 KIRKWOOD AVE,OSAGE,IA,50461,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SIOUX COUNTY EGG FARM LLP,2716 380TH ST,SIOUX CENTER,IA,51250,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514404.0,POULTRY-WHOLESALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,E & W POULTRY FARM,2618 340TH ST,ROCK VALLEY,IA,51247,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SUNRISE FARMS INC,2060 WHITE AVE,HARRIS,IA,51345,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,FREMONT FARMS,3205 295TH ST,FREMONT,IA,52561,514101,FOOD PRODUCTS (WHLS),42442005,PACKAGED FROZEN FOOD MERCHANT WHOLESALERS,514403.0,EGGS (WHLS),999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,FARMERS HEN HOUSE,1956 520TH ST SW,KALONA,IA,52247,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
igroup_sic_5144_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,JOHNSON COUNTY EGG FARM,1275 SE Y HWY,KNOB NOSTER,MO,65336,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KNOB NOSTER PULLETS,1251 SE Y HWY,KNOB NOSTER,MO,65336,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JOHNSON COUNTY EGG FARM,1285 SE Y HWY,KNOB NOSTER,MO,65336,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LEUSCHEN POULTRY OPERATIONS,1175 DAIRY CIR,CRANE,MO,65633,514402,POULTRY SERVICES (WHLS),42444002,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MOARK PRODUCTION,17331 HIGHWAY D,NEOSHO,MO,64850,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MOARK PRODUCTION INC,19242 MALLARD DR,NEOSHO,MO,64850,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
igroup_sic_5144_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,KOCH FOODS INC,764 GEORGE CAGLE DR,COLLINSVILLE,AL,35961,201501,POULTRY PROCESSING PLANTS (MFRS),31161501,POULTRY PROCESSING,514402.0,POULTRY SERVICES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,517549762.0


In [84]:
igroup_sic_5144_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TOTAL POULTRY SVC LLC,130 STOVALL RD # 2,KOKOMO,MS,39643,514402,POULTRY SERVICES (WHLS),42444002,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
igroup_sic_5144_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,WABASH VALLEY PRODUCE INC,4886 E 450N,DUBOIS,IN,47527,204803,FEED-MANUFACTURERS,31111903,OTHER ANIMAL FOOD MANUFACTURING,519112.0,FEED-DEALERS (WHLS),514403.0,EGGS (WHLS),549914.0,EGGS-RETAIL,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN


In [86]:
igroup_sic_5144_dict['KY']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,CAL-MAINE FOODS INC,9729 GUTHRIE RD,GUTHRIE,KY,42234,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,431481290.0


In [88]:
igroup_sic_5144_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SIMPSON'S EGGS INC,5015 E HIGHWAY 218,MONROE,NC,28110,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
igroup_sic_5144_dict['GA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,CLAXTON POULTRY FARMS,8816 US HIGHWAY 301,CLAXTON,GA,30417,201501,POULTRY PROCESSING PLANTS (MFRS),31161501,POULTRY PROCESSING,25401.0,POULTRY FARMS,509901.0,EXPORTERS (WHLS),514404.0,POULTRY-WHOLESALE,514403.0,EGGS (WHLS),NaN,NaN,NaN


## SIC Code 5154 Exploration: Livestock

In [94]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 5154
df_sic_5154 = pd.read_csv("../data/code_5154.csv")
  
igroup_sic_5154_dict, ctrglow_5154_dict = infogroup_counterglow_dict(df_sic_5154, df_counterglow_poultry, states, RADIUS_KM)


In [91]:
# igroup_sic_5154_dict = keep_certain_columns(igroup_sic_5154_dict)

In [95]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_5154_dict, all_df_list)

#### Explore

In [93]:
igroup_sic_5154_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,ROCKING W LIVESTOCK EQPT SALES,600 N JOHNSON CHAPEL RD,DANVILLE,AL,35619,515403,LIVESTOCK-DEALERS (WHLS),42452008,LIVESTOCK MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MID STATE STOCKYARDS LLP,8415 STATE HIGHWAY 97 S,LETOHATCHEE,AL,36047,478904,STOCK YARDS,48821007,SUPPORT ACTIVITIES FOR RAIL TRANSPORTATION,515402.0,LIVESTOCK AUCTION MARKETS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIC Code 0251: Broiler, Fryer, and Roaster Chickens

In [96]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0251
df_sic_0251 = pd.read_csv("../data/code_0251.csv")
  
igroup_sic_0251_dict, ctrglow_0251_dict = infogroup_counterglow_dict(df_sic_0251, df_counterglow_poultry, states, RADIUS_KM)


NC: Total number of matches: 1


In [97]:
# igroup_sic_0251_dict = keep_certain_columns(igroup_sic_0251_dict)

In [97]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(igroup_sic_0251_dict, all_df_list)

In [107]:
select_df_list = list_of_dataframes(igroup_sic_0251_dict, select_df_list)

#### Explore

In [98]:
igroup_sic_0251_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TRIPLE P RANCH & EQUIPMENT LLC,22211 FIVE BRIDGES AVE,COLE CAMP,MO,65325,25198,BROILER FRYER & ROASTER CHICKENS,11232001,BROILERS & OTHER MEAT TYPE CHICKEN PRODUCTION,508303.0,POULTRY EQUIPMENT & SUPPLIES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
igroup_sic_0251_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,WOODHAM FARM,2501 COUNTY ROAD 53,CLAYTON,AL,36016,25298,CHICKEN EGGS,11231001,CHICKEN EGG PRODUCTION,25198.0,BROILER FRYER & ROASTER CHICKENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
igroup_sic_0251_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,REYNOLDS FARMS,7518 CARL POLK RD,MONROE,NC,28110,25198,BROILER FRYER & ROASTER CHICKENS,11232001,BROILERS & OTHER MEAT TYPE CHICKEN PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## To CSV

In [98]:
### concat all DataFrames & convert to csv

infogroup_concat = pd.concat(all_df_list)

FILEPATH = "../data/infogroup_matches.csv"
infogroup_concat.to_csv(FILEPATH) # save as CSV file

In [108]:
# concat the select dataframes and convert to csv

infogroup_concat_selected = pd.concat(select_df_list)

FILEPATH = "../data/infogroup_matches_selects.csv"
infogroup_concat_selected.to_csv(FILEPATH)